In [1]:
import importlib

import os
import sys
import json
import warnings

import numpy as np

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

import plotting as plot
import transcription
from utilities import *
from signal_processing import *

from scipy.io.wavfile import write as wav_write

In [ ]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

In [ ]:
indices = np.random.randint(0, len(track_titles), 40)

In [ ]:
for idx in indices:
    
    try:

        title = track_titles[idx] 
        BPM = float(track_dicts[title]['BPM'])
        beat_length = 60/BPM

        quarter_beat_positions = get_quarter_beat_positions(get_progression_beat_positions(title))

        if title in bad_examples:
            continue
            with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
                dpqb = float(infile.read())
            print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))

        track_scale = get_track_scale(title, track_dicts, scales)

        fs = 44100
        chorus, bassline, unprocessed_bassline = load_audio(title) # read the chorus and the bassline
        
        spectrogram_beat_factor = 8

        win_length = int((beat_length/spectrogram_beat_factor)*fs) 
        n_fft = 4096*8

        hop_length = int(win_length/2) 
        assert win_length < n_fft, 'Window length must be greater than N_fft'

        bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)
        
        
        beat_factor = 8 # for frame size quarter 4, 1/8th 8...

        # initial estimate and the filtered version
        F0_estimate, pitch_track = transcription.pYIN_F0(bassline,
                                                       fs,
                                                       int((beat_length/beat_factor)*fs),
                                                       threshold=0.05)   
        
        # Quantize the pitch track
        pitch_track_quantized = transcription.adaptive_voiced_region_quantization(pitch_track,
                                                                    track_scale,
                                                                    quarter_beat_positions, 
                                                                    length_threshold=beat_factor,
                                                                    epsilon=2)

        # Extract the Notes for plotting
        bassline_notes, unk_bassline_notes = transcription.extract_notes(pitch_track_quantized, track_scale) 

        # map frequencies to midi
        bassline_midi_notes = transcription.get_midi_notes(pitch_track_quantized, 16*beat_factor)
        
        midi_path = '../data/midi_notes/numpy/{}'.format(title)
        np.save('{}.npy'.format(midi_path), bassline_midi_notes)
        
        wav_write('../data/audio_outputs/choruses/{}.wav'.format(title), fs, chorus)
        
    except:
        print(idx)